In [51]:
#importing required api and requests
from fastapi import FastAPI
import requests
import sqlite3

In [37]:
app = FastAPI()

# initializing database
DB_NAME = "jokes.db"

In [41]:
def init_db():

    
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("""

    
        CREATE TABLE IF NOT EXISTS jokes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            
            category TEXT,
            type TEXT,
            joke TEXT,
            setup TEXT,
            delivery TEXT,
            nsfw BOOLEAN,
            political BOOLEAN,
            sexist BOOLEAN,
            safe BOOLEAN,
            lang TEXT
        )
    """)
    conn.commit()
    conn.close() 
init_db

<function __main__.init_db()>

In [43]:
#  fetching  jokes from external API function
def fetch_jokes(count=100):
    url = f"https://v2.jokeapi.dev/joke/Any?amount={count}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json().get("jokes", [])
    return []

In [53]:
#save jokes to the database
def save_jokes(jokes):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    
    for joke in jokes:
        category = joke.get("category", "")
        
        
        joke_type = joke.get("type", "")
        joke_text = joke.get("joke", "") if joke_type == "single" else ""
        
        setup = joke.get("setup", "") if joke_type == "twopart" else ""
        delivery = joke.get("delivery", "") if joke_type == "twopart" else ""
        nsfw = joke["flags"]["nsfw"]
        
        political = joke["flags"]["political"]
        sexist = joke["flags"]["sexist"]
        safe = joke.get("safe", False)
        lang = joke.get("lang", "")

        cursor.execute("""
            INSERT INTO jokes (category, type, joke, setup, delivery, nsfw, political, sexist, safe, lang)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (category, joke_type, joke_text, setup, delivery, nsfw, political, sexist, safe, lang))

    conn.commit()
    conn.close()

In [55]:
@app.get("/fetch_and_store_jokes")
def fetch_and_store():
    jokes = fetch_jokes(100)
    if jokes:
        save_jokes(jokes)
        return {"message": "Jokes successfully fetched and stored", "total_jokes": len(jokes)}
    return {"message": "Failed to fetch jokes"}

In [57]:
@app.get("/get_jokes")
def get_jokes():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM jokes LIMIT 10")
    jokes = cursor.fetchall()
    conn.close()
    return {"jokes": jokes}

SyntaxError: invalid non-printable character U+00A0 (1143904632.py, line 8)